In [4]:
import os

from langchain_community.graphs import Neo4jGraph

os.environ["NEO4J_URI"] = "bolt://localhost:7687"
os.environ["NEO4J_USERNAME"] = "neo4j"
os.environ["NEO4J_PASSWORD"] = "2wsx3edc"

graph = Neo4jGraph()

In [5]:
graph.query(
    """
MATCH (n:`__Community__`)<-[:IN_COMMUNITY]-()<-[:HAS_ENTITY]-(c)
WITH n, count(distinct c) AS chunkCount
SET n.weight = chunkCount"""
)

[]

In [6]:
# from langchain_openai import AzureOpenAIEmbeddings

# embedding = AzureOpenAIEmbeddings(
#     model="text-embedding-ada-002",
#     azure_endpoint='https://lang-chain-dev.openai.azure.com/openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-05-15',
#     azure_deployment='text-embedding-ada-002',
#     openai_api_version='2023-05-15'
# )

In [7]:
from langchain_community.vectorstores import Neo4jVector
# ! pip3 install -U langchain-huggingface
import os
os.environ['SENTENCE_TRANSFORMERS_HOME'] = '/storage/models/embedding_models'
from langchain_huggingface import HuggingFaceEmbeddings
# Choose from https://huggingface.co/spaces/mteb/leaderboard
embedding = HuggingFaceEmbeddings(model_name="lier007/xiaobu-embedding-v2")

/storage/pyenv/versions/3.12.5/envs/ACP_LLM_312/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/storage/pyenv/versions/3.12.5/envs/ACP_LLM_312/lib/python3.12/site-packages/sentence_transformers/models/Dense.py:89: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded v

In [8]:
from langchain_community.vectorstores import Neo4jVector

lc_retrieval_query = """
WITH collect(node) as nodes
// Entity - Text Unit Mapping
WITH
collect {
    UNWIND nodes as n
    MATCH (n)<-[:HAS_ENTITY]->(c:__Chunk__)
    WITH c, count(distinct n) as freq
    RETURN c.content AS chunkText
    ORDER BY freq DESC
    LIMIT $topChunks
} AS text_mapping,
// Entity - Report Mapping
collect {
    UNWIND nodes as n
    MATCH (n)-[:IN_COMMUNITY]->(c:__Community__)
    WITH c, c.rank as rank, c.weight AS weight
    RETURN c.summary 
    ORDER BY rank, weight DESC
    LIMIT $topCommunities
} AS report_mapping,
// Outside Relationships 
collect {
    UNWIND nodes as n
    MATCH (n)-[r]-(m) 
    WHERE NOT m IN nodes
    RETURN r.description AS descriptionText
    ORDER BY r.rank, r.weight DESC 
    LIMIT $topOutsideRels
} as outsideRels,
// Inside Relationships 
collect {
    UNWIND nodes as n
    MATCH (n)-[r]-(m) 
    WHERE m IN nodes
    RETURN r.description AS descriptionText
    ORDER BY r.rank, r.weight DESC 
    LIMIT $topInsideRels
} as insideRels,
// Entities description
collect {
    UNWIND nodes as n
    RETURN n.description AS descriptionText
} as entities
// We don't have covariates or claims here
RETURN {Chunks: text_mapping, Reports: report_mapping, 
       Relationships: outsideRels + insideRels, 
       Entities: entities} AS text, 1.0 AS score, {} AS metadata
"""

vectorstore = Neo4jVector.from_existing_graph(embedding=embedding, 
                                    index_name="description_embedding",
                                    node_label='__Entity__', 
                                    embedding_node_property='description_embedding', 
                                    text_node_properties=['description'],
                                    retrieval_query=lc_retrieval_query)

In [9]:
topChunks = 3
topCommunities = 3
topOutsideRels = 10
topInsideRels = 10
topEntities = 10

In [10]:
os.environ["NEO4J_URI"] = "bolt://localhost:7687"
os.environ["NEO4J_USERNAME"] = "neo4j"
os.environ["NEO4J_PASSWORD"] = "2wsx3edc"

In [11]:
docs = vectorstore.similarity_search(
    "你對台灣人壽知道多少",
    k=topEntities,
    params={
        "topChunks": topChunks,
        "topCommunities": topCommunities,
        "topOutsideRels": topOutsideRels,
        "topInsideRels": topInsideRels,
    },
)
print(docs[0].page_content)

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownRelationshipTypeWarning} {category: UNRECOGNIZED} {title: The provided relationship type is not in the database.} {description: One of the relationship types in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing relationship type is: RELATED)} {position: line: 25, column: 18, offset: 679} for query: "CALL db.index.vector.queryNodes($index, $k, $embedding) YIELD node, score \nWITH collect(node) as nodes\n// Entity - Text Unit Mapping\nWITH\ncollect {\n    UNWIND nodes as n\n    MATCH (n)<-[:HAS_ENTITY]->(c:__Chunk__)\n    WITH c, count(distinct n) as freq\n    RETURN c.content AS chunkText\n    ORDER BY freq DESC\n    LIMIT $topChunks\n} AS text_mapping,\n// Entity - Report Mapping\ncollect {\n    UNWIND nodes as n\n    MATCH (n)-[:IN_COMMUNITY]->(c:__Community__)\

Entities:
- 依各險種投保規則辦理
- 提供保险服务的公司
- 购买保险并持有保险合同的人
- 首次需繳交的保險費用
- 不得為空白授權，應依是否取得保單號碼填寫相關欄位
- 加強身分驗證機制
- 通過銀行或金融机构進行转账支付的作业方式
- 从客户的活期账户中自动扣除费用
Reports:
- 此同社區的資訊描述了一家提供保險服務的公司，該公司在付款方式上使用了通過銀行或金融機構進行的轉賬支付。首期的保險費用是通過這種金融機構轉帳作業方式來繳交的。
- 根據提供的資訊，保單持有人會從他們的活期賬戶中自動扣除相關費用。
- None
Chunks:
-                                                                 回目錄   6  版次： 113年07月版                                                                                            修訂日期： 113.0 7.01 參、  首期保險費繳交方式及相關規定   一、  首期保險費 -採金融機構 轉帳作業  (一) 首期保險費採 金融機構轉帳作業 應注意事項   1. 首期保險費由保戶 約定於本公司 簽約之金融機構所開立之活期帳戶扣款 
- ， 保費折扣適用規定依各險種投保規則辦理。   8. 新契約投保時授權書 不得為空白授權， 應依是否取得保單號碼 填寫相關欄位 ，未填寫完 整須重新檢附 授權書。 相關規定請參照 自動轉帳及信用卡付款授權書作業規範 (加強身分 驗證機制 )。
Relationships:



In [12]:
docs

[Document(page_content='Entities:\n- 被保險人的年齡\n- 台灣人壽 美鑫美利美元利率變動型終身壽險\n- 前項未滿15足歲之被保險人如有於民國九十九年二月三日前（不含）訂立之保險契約，其喪葬費用保險金之給付依下列方式辦理。\n- 保險費的繳費期間\n- 身故保險金或喪葬費用保險金的給付。\n- 保單的第六個年度\n- 保單的週年紀念日\n- 被保險人於本契約有效期間內身故者，本公司按身故日之當年度保險金額給付身故保險金。\n- 選擇儲存生息者，各年度之增值回饋分享金將按各保單週年日當月之宣告利率依據年複利方式，累積至要保人請求時給付，或至被保險人身故或本契約終止時，由本公司主動一併給付。惟須於第六保單年度屆滿後之每一保單週年日起，始得依本款方式給付\n- 訂立本契約時，以未滿15足歲之未成年人為被保險人，除喪葬費用之給付外，其餘死亡給付之約定於被保險人滿15足歲之日起發生效力；被保險人滿15足歲前死亡者，其身故保險金變更為喪葬費用保險金。\nReports:\n- 台灣人壽提供一款名為「美鑫美利美元利率變動型終身壽險」的保險商品。這款保險商品的詳細內容可以通過其說明文件了解。\n- 在這個社區中，描述了一個與保險相關的情境。具體來說，若被保險人在民國九十九年二月三日（不含）之前訂立了保險契約，則其喪葬費用保險金的給付將依特定方式辦理。這一日期被標記為一個重要的時間點，並且與保險契約存在簽訂關係。\n- None\nChunks:\n- ，以書面通知 本公司變更前項給付方式，惟第六保單年度屆滿前，增值回饋分享金限以購買增額繳清保險金額 辦理。  被保險人保險年齡到達 16歲前，其增值回饋分享金於繳費期間應採抵繳保險費之方式辦理。但 因繳費期間已屆滿而無法抵繳保險費者，本公司改按各保單週年日當月之宣告利率，以年複利方 式儲存生息至被保險人保險年齡到達 16歲時，以累計儲存生息之金額一次計算增額繳清保險金 額，其後保單年度適用第一項規定。   要保人終止本契約，或被保險人於保險年齡達 16歲前死亡者，本公司應退還歷年累計儲存生息 之金額予要保人。   本公司於每一保 單年度屆滿後，應將該增值回饋分享金之金額\n- ，應將該增值回饋分享金之金額，以書面或電子郵件方式通知要保 人。    【身故保險金或喪葬費用保險金的給付】   第十六條